<a href="https://colab.research.google.com/github/iyoo2018/findatalake/blob/master/scrapeRSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Exclusive to Colab

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
  from google.colab import drive
  drive.mount('/content/gdrive')
  import sys
  sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

Mounted at /content/gdrive


In [45]:
if 'COLAB_GPU' in os.environ:
  os.environ['AWS_CONFIG_FILE']="/content/gdrive/My Drive/cred-stockdata.txt"
  os.environ["bucket"] = "026090555438-stockdata"
  os.environ["rssKey"] = "rssList.json"

In [ ]:
"""
# CNBC Top News
topNews = {"network":"CNBC", "feed":"Top News", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=100003114"}
# Earnings
earning = {"network":"CNBC", "feed":"Earnings", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=15839135"}
# Economy
economy = {"network":"CNBC", "feed":"Economy", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=20910258"}
# Finance
finance = {"network":"CNBC", "feed":"Finance", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000664"}
# Tech
tech = {"network":"CNBC", "feed":"Tech", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910"}
# Investing
invest = {"network":"CNBC", "feed":"Investing", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=15839069"}
"""

'\n# CNBC Top News\ntopNews = {"network":"CNBC", "feed":"Top News", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=100003114"}\n# Earnings\nearning = {"network":"CNBC", "feed":"Earnings", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=15839135"}\n# Economy\neconomy = {"network":"CNBC", "feed":"Economy", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=20910258"}\n# Finance\nfinance = {"network":"CNBC", "feed":"Finance", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000664"}\n# Tech\ntech = {"network":"CNBC", "feed":"Tech", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=19854910"}\n# Investing\ninvest = {"network":"CNBC", "feed":"Investing", "url":"https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=15839069"}\n'

# Import Packages

In [46]:
import feedparser
import requests
from bs4 import BeautifulSoup
import json
import re
import boto3
import datetime

# Scrape RSS Feeds

In [47]:
# Fetch the entries from an RSS feed
# Arg: source [string] *contains RSS feed link*
# Returns: entries [list of dictionaries] *contains entries in RSS feed*
def fetchRSS(source):
  # fetch the RSS feed
  RSS = feedparser.parse(source)
  # extract the entries from the retrieved feed
  entries = RSS.entries
  return entries

# Parse HTML

In [48]:
# Scrape an entry using the url
# Arg: url [string] **contains url for the entry**
# Returns: soup [BeautifulSoup object]
def scraper(url):
  # scrape the data from the url using requests and BeautifulSoup
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  return soup

# Extract text from the scraped/parsed entry
# Arg: soup [BeautifulSoup object]
# Returns: result [string] **contains the text of the entry**
def textExtract(soup):
  # extract the data corresponding to the entry from the soup
  entryGroups = soup.find_all("div", {"class": "group"})
  sections = []
  for group in entryGroups:
    section = group.find_all("p")
    if section:
      sections.append(section)
  # extract the text of the entry
  text = []
  for section in sections:
    for para in section:
      text.append(para.get_text())
  # return the entry joined as a single string
  return ("\n").join(text)

# Get publication dates

In [59]:
# Get publication dates of files
# Arg: bucket **bucket name** [str],
#      keys **list of keys that require dates** [list of str]
# Returns: list of dates [list of str]
def getDates(bucket, keys, s3Helper):
  dates = []
  for key in keys:
    date = (json.loads(s3Helper.getObj(bucket, key)['Body'].read().decode()))
    dates.append(date["date"])
  return dates

# Convert publication dates

In [60]:
# Convert publication dates of files
# Arg: unstrDates **unstructed dates** [list of str]
# Returns: list of structured dates [list of str]
def convertDates(unstrDates):
  dateFormat = "%a, %d %b %Y %H:%M:%S %Z"
  return [datetime.datetime.strptime(unstrDate, dateFormat) for unstrDate in unstrDates]

# AccessS3 Class

In [51]:
class AccessS3:
  def __init__(self):
    session = boto3.Session()
    self.s3 = session.client('s3')
    self.paginator = self.s3.get_paginator('list_objects_v2')

  # Get an object
  # Arg: bucket **bucket name** [str],
  #      key **object key** [str],
  # Returns: object
  def getObj(self, bucket, key):
    return self.s3.get_object(Bucket=bucket, Key=key)

  # Delete an object
  # Arg: bucket **bucket name** [str],
  #      key **object key** [str]
  def deleteObj(self, bucket, key):
    self.s3.delete_object(Bucket=bucket, Key=key)
    print("Deleted object at {}".format(key))
    return 0

  # Save an object
  # Arg: data **data to be saved**
  #      bucket **bucket name** [str],
  #      key **object key** [str]
  def saveObj(self, data, bucket, key):
    self.s3.put_object(
      Body=data,
      Bucket=bucket,
      Key=key
    )
    print("Saved object at {}".format(key))
    return 0

  # Look at objects contained in a key
  # Arg: bucket **bucket name** [str],
  #      key **object key** [str]
  # Returns: objs **objects in key**
  def scanFolder(self, bucket, key):
    objs = []
    pages = self.paginator.paginate(Bucket=bucket, Prefix=key)
    for page in pages:
      for content in page['Contents']:
        if not content['Key'].endswith("/"):
          objs.append(content['Key'])
    return objs

  # Look up an object
  # Arg: bucket [string] **S3 bucket to look in**
  #      key [string] **key to look in**
  #      id [str] **lookup object id**
  # Returns: key **object key if it exists**
  def lookupObj(self, bucket, key, query, group):
    keys = []
    objs = self.scanFolder(bucket, key)
    for obj in objs:
      lookup = re.search(query, obj)
      if lookup:
        keys.append(lookup.group(group))
    return keys

# SaveStockData Class

In [63]:
class SaveStockData:
  def __init__(self, s3Helper):
    self.s3Helper = s3Helper

  def createMetaPath(self, newEntry, network):
    id = newEntry["id"]
    year = newEntry["published_parsed"][0]
    month = newEntry["published_parsed"][1]
    day = newEntry["published_parsed"][2]
    metaKey = "metadata/{}/{}/{}/{}/{}.json".format(network, year, month, day, id)
    return metaKey

  def createTextPath(self, newEntry, network):
    id = newEntry["id"]
    year = newEntry["published_parsed"][0]
    month = newEntry["published_parsed"][1]
    day = newEntry["published_parsed"][2]
    textKey = "textdata/{}/{}/{}/{}/{}.json".format(network, year, month, day, id)
    return textKey

  def collectMetaData(self, newEntry, feed):
    title = newEntry["title"]
    link = newEntry["link"]
    date = newEntry["published"]
    sponsor = newEntry["metadata_sponsored"]
    metaType = newEntry["metadata_type"]
    metaData = json.dumps({"title":title,"link":link,"date":date,"feed":feed,"sponsor":sponsor,"type":metaType})
    return metaData

  def collectTextData(self, newEntry):
    textData = json.dumps(newEntry["text"])
    return textData

  def saveData(self, data, bucket, key):
    self.s3Helper.saveObj(data, bucket, key)
    return 0

# Save New Entries to S3

In [64]:
# Save the text data and metadata for a new entry
# Arg: newEntry [dict] **new entry to be saved**
#      network [string] **network that published the RSS feed**
#      feed [string] **name of the RSS feed**
#      bucket [string] **S3 bucket where data is stored**
def saveSingleEntry(newEntry, network, feed, bucket, s3Helper):
  stockSaver = SaveStockData(s3Helper)

  # scrape and add the text to the entry
  entryURL = newEntry["link"]
  soup = scraper(entryURL)
  result = textExtract(soup)
  newEntry["text"] = result

  # create object path
  metaKey = stockSaver.createMetaPath(newEntry, network)
  textKey = stockSaver.createTextPath(newEntry, network)

  # collect metadata and text data
  metaData = stockSaver.collectMetaData(newEntry, feed)
  textData = stockSaver.collectTextData(newEntry)

  # upload metadata
  stockSaver.saveData(metaData, bucket, metaKey)
  # upload text data
  stockSaver.saveData(textData, bucket, textKey)

  return 0

In [65]:
# Save new entries to S3 and update old ones
# Arg: entries [list of dictionaries] **entries gathered from feed**
#      network [string] **network that published the RSS feed**
#      feed [string] **name of the RSS feed**
#      bucket [string] **S3 bucket where data is stored**
#      folder [string] **folder where data is stored**
def saveNewEntries(entries, network, feed, bucket):
  s3Helper = AccessS3()
  count = 0
  print("This feed has {} entries".format(len(entries)))
  # get the existing entries from s3
  existKeys = s3Helper.scanFolder(bucket, "metadata")
  for entry in entries:
    # get the entry id
    entryID = entry["id"]
    # if the entry id does not already exist
    if not any(entryID in existKey for existKey in existKeys):
      # then save it to s3
      print("Saving new entry id {}:".format(id))
      saveSingleEntry(entry, network, feed, bucket, s3Helper)
      count += 1
    # otherwise it already exists
    else:
      # search for matching entries
      matchKeys = [existKey.split("/", 1)[1] for existKey in existKeys if entryID in existKey]
      metaKeys = ["metadata/"+matchKey for matchKey in matchKeys]
      textKeys = ["textdata/"+matchKey for matchKey in matchKeys]
      # retrieve and convert dates to comparable datetime objects
      matchDates = getDates(bucket, metaKeys, s3Helper)
      existDates = convertDates(matchDates)
      latestDate = max(existDates)
      entryDate = convertDates([entry["published"]])[0]
      # if the entry has been updated,
      if entryDate > latestDate:
        print("Updating entry id: {}".format(id))
        for metaKey, textKey in zip(metaKeys, textKeys):
          # delete the old one(s)
          #print("Deleted {}".format(metaKey))
          #print("Deleted {}".format(textKey))
          s3Helper.deleteObj(bucket, metaKey)
          s3Helper.deleteObj(bucket, textKey)
        # and save the new one
        saveSingleEntry(entry, network, feed, bucket, s3Helper)
        count += 1
      # otherwise check for duplicates
      elif len(matchKeys) > 1:
        print("Deleting duplicate entries id: {}".format(id))
        for metaKey, textKey, existDate in zip(metaKeys, textKeys, existDates):
          if not existDate==latestDate:
            # and delete the older ones
            #print("Deleted {}".format(metaKey))
            #print("Deleted {}".format(textKey))
            s3Helper.deleteObj(bucket, metaKey)
            s3Helper.deleteObj(bucket, textKey)
  # report the total number of entries that have been saved
  if count==0:
    print('No new entries for {}'.format(feed))
  else:
    print("{} new entries for {}".format(count, feed))
  return 0

# main

In [66]:
def main(event, context):
  bucket = os.environ["bucket"]
  rssKey = os.environ["rssKey"]
  s3Helper = AccessS3()
  # Retrieve list of RSS feeds to gather entries from
  obj = s3Helper.getObj(bucket, rssKey)
  rssList = json.loads(obj["Body"].read().decode())
  # Gather entries from the feeds and save new ones to S3
  count = 0
  for rss in rssList:
    rssNetwork = rss["network"]
    rssFeed = rss["feed"]
    rssURL = rss["url"]
    # Fetch entries from the current RSS feed
    print("Currently scraping from {}".format(rssFeed))
    entries = fetchRSS(rssURL)
    # Save new entries to S3
    saveNewEntries(entries, rssNetwork, rssFeed, bucket)
  return {
      'statusCode': 200
  }

# Test

In [67]:
if 'COLAB_GPU' in os.environ:
  # import text and test
  result = main(None, None)
  print(result)

Currently scraping from Top News
This feed has 30 entries
No new entries for Top News
Currently scraping from Earnings
This feed has 30 entries
No new entries for Earnings
Currently scraping from Economy
This feed has 30 entries
No new entries for Economy
Currently scraping from Finance
This feed has 30 entries
No new entries for Finance
Currently scraping from Tech
This feed has 30 entries
No new entries for Tech
Currently scraping from Investing
This feed has 30 entries
No new entries for Investing
{'statusCode': 200}
